# Run Blob or Peak Detection on Pixel Heatmap to Identify Candidate Sites
Note: This is only working on inputs with EPSG CRS 4326. I may need to make it more general in the future.

In [ ]:
import json
import geojson
from keplergl import KeplerGl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import rasterio as rs
from rasterio.windows import Window
from rasterio import warp
from skimage.feature import blob_doh
from skimage.feature.peak import peak_local_max
from sklearn.neighbors import KDTree

In [ ]:
def pixels_to_coords(x, y, src_img):
    lon, lat = warp.transform(src_img.crs, rs.crs.CRS.from_epsg(4326), [src_img.xy(x, y)[0]], [src_img.xy(x, y)[1]])
    return lon[0], lat[0]

def coords_to_pixels(lon, lat, src_img):
    x, y = warp.transform(rs.crs.CRS.from_epsg(4326), src_img.crs, [lon],  [lat])
    pixel_y, pixel_x = src_img.index(x, y)
    return pixel_x[0], pixel_y[0]

def merge_similar_sites(candidate_sites, search_radius=0.01):
    """
    This process iteratively moves points closer together by taking the mean position of all
    matched points. It then searches the KD tree using the unique clusters in these new points.
    The algorithm stops once the number of unique sites is the same as in the previous round.
    
    search_radius is given in degrees
    """
    #coords = np.array([[lon, lat] for lon, lat in zip(candidate_site_df['lon'], candidate_site_df['lat'])])
    coords = np.array(candidate_sites)
    
    # Create a KD tree for efficient lookup of points within a radius
    tree = KDTree(coords, leaf_size=2)
    
    # Initialize a mean_coords array for the search
    mean_coords = []
    for elem in tree.query_radius(coords, search_radius):
        mean_coords.append(np.mean(coords[elem], axis=0))
    mean_coords = np.array(mean_coords)
    
    num_coords = len(mean_coords)
    while True:
        search = tree.query_radius(mean_coords, search_radius)
        uniques = [list(x) for x in set(tuple(elem) for elem in search)]
        mean_coords = []
        for elem in uniques:
            mean_coords.append(np.mean(coords[elem], axis=0))
        if len(mean_coords) == num_coords:
            print(len(mean_coords), "unique sites detected")
            mean_coords = np.array(mean_coords)
            break
        num_coords = len(mean_coords)
        
    unique_sites = pd.DataFrame(mean_coords, columns=['lon', 'lat'])
    unique_sites['name'] = [f"{name}_{i+1}" for i in range(len(unique_sites))]
    plt.figure(figsize=(10,10), dpi=150, facecolor=(1,1,1))
    plt.scatter(coords[:,0], coords[:,1], s=5, label='Original')
    plt.scatter(mean_coords[:,0], mean_coords[:,1], s=3, c='r', label='Unique')
    plt.axis('equal')
    plt.legend()
    plt.show()
    
    return unique_sites

def detect_peaks(source, name, threshold_rel=0.75, min_distance=10, window_size=5000, save=True):
    candidate_peaks = []
    for x in range(0, source.shape[0], window_size):
        for y in range(0, source.shape[1], window_size):
            window = Window.from_slices((x,x + window_size), (y, y + window_size))
            transform = source.window_transform(window)
            subset = source.read(1, window=window)
            peaks = peak_local_max(subset, threshold_rel=threshold_rel, min_distance=min_distance)
            for candidate in peaks:
                lon, lat = (transform * [candidate[1], candidate[0]])
                candidate_peaks.append([lon, lat])
    print(len(candidate_peaks), "peaks detected")
    
    if save:
        file_path = f"../data/model_outputs/candidate_sites/{name}_peaks_thresh_{threshold_rel}_min_dist_{min_distance}"
        geojson_features = []
        for i, (lon, lat) in enumerate(candidate_peaks):
            geojson_features.append(geojson.Feature(geometry = geojson.Point([lon, lat]),
                                                    properties={'name': f"{name}_{i+1}"}))
        feature_collection = geojson.FeatureCollection(geojson_features)
        with open(file_path + '.geojson', 'w') as f:
            f.write(geojson.dumps(feature_collection))
    
    candidate_peak_df = pd.DataFrame(candidate_peaks, columns=['lon', 'lat'])
    candidate_peak_df['name'] = [f"{name}_{i}" for i in range(len(candidate_peaks))]
    return candidate_peak_df

def detect_blobs(source, name, pred_threshold=0.75, min_sigma=3.5, max_sigma=100, area_threshold=0.0025, window_size=5000, save=True):
    """
    Identify candidates using blob detection on the heatmap.
    prediction_threshold masks any prediction below a 0-1 threshold.
    min_sigma and area_threshold control the size sensitivity of the blob detection.
    Keep min_sigma low to detect smaller blobs
    area_threshold establishes a lower bound on candidate blob size. Reduce to detect smaller blobs
    """
    candidate_sites = []
    max_val = source.read(1).max()
    for x in range(0, source.shape[0], window_size):
        for y in range(0, source.shape[1], window_size):
            print(f"Processing row {(x // window_size) + 1} of {int(source.shape[0] / window_size) + 1}, column {(y // window_size) + 1} of {int(source.shape[1] / window_size) + 1}")
            # Set min and max to analyze a subset of the image
            window = Window.from_slices((x,x + window_size), (y, y + window_size))
            window_median = (source.read(1, window=window) / max_val).astype('float')
            # mask predictions below a threshold
            mask = np.ma.masked_where(window_median < pred_threshold, window_median).mask
            window_median[mask] = 0

            blobs = blob_doh(window_median, min_sigma=min_sigma, max_sigma=max_sigma, threshold=area_threshold)
            print(len(blobs), "candidates detected in window")
            
            overlap_threshold = 0.01
            transform = source.window_transform(window)
            for candidate in blobs:
                lon, lat = (transform * [candidate[1], candidate[0]])
                # Size doesn't mean anything at the moment. Should look into this later
                #size = candidate[2]
                candidate_sites.append([lon, lat])
            
    print(len(candidate_sites), "candidate sites detected in total")
    
    candidate_site_df = merge_similar_sites(candidate_sites, search_radius=0.01)
    
    if save:
        # Write candidates to CSV and GeoJSON
        file_path = f"../data/model_outputs/candidate_sites/{name}_candidates_pred-thresh_{pred_threshold}_min-sigma_{min_sigma}_area-thresh_{area_threshold}"
        candidate_site_df.to_csv(file_path + '.csv', index = False)
        print("Candidate sites written to", file_path)

        geojson_features = []
        for i, (lon, lat) in enumerate(zip(candidate_site_df['lon'], candidate_site_df['lat'])):
            geojson_features.append(geojson.Feature(geometry = geojson.Point([lon, lat]),
                                                    properties={'name': f"{name}_{i+1}"}))
        feature_collection = geojson.FeatureCollection(geojson_features)
        with open(file_path + '.geojson', 'w') as f:
            f.write(geojson.dumps(feature_collection))
    
    return candidate_site_df

In [ ]:
kepler_config = {
  "version": "v1",
  "config": {
    "visState": {
      "layers": [
        {
          "id": "iik903a",
          "type": "point",
          "config": {
            "dataId": "Candidate Peaks",
            "label": "Point",
            "color": [
              218,
              0,
              0
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 20,
              "fixedRadius": False,
              "opacity": 0.99,
              "outline": True,
              "thickness": 3,
              "strokeColor": [
                210,
                0,
                0
              ],
              "filled": False
            },
          },
        },
        {
          "id": "kyoc7uj",
          "type": "point",
          "config": {
            "dataId": "Candidate Blobs",
            "label": "Point",
            "color": [
              246,
              218,
              0
            ],
            "columns": {
              "lat": "lat",
              "lng": "lon",
              "altitude": None
            },
            "isVisible": True,
            "visConfig": {
              "radius": 20,
              "fixedRadius": False,
              "opacity": 0.99,
              "outline": True,
              "thickness": 3,
              "strokeColor": [
                246,
                218,
                0
              ],
              "filled": False
            },
          },
        }
      ],
    },
    "mapStyle": {
      "styleType": "satellite",
    }
  }
}

In [ ]:
candidate_map = KeplerGl(height=800, config=kepler_config)

In [ ]:
name = 'Bali_v1.1.8_2019-2020'
source = rs.open(f'../data/model_outputs/heatmaps/{name}.tif')

In [ ]:
peaks = detect_peaks(source, name=name, min_distance=100, threshold_rel=0.9, save=True)
candidate_map.add_data(data=peaks, name='Candidate Peaks')

In [ ]:
blobs = detect_blobs(source, name=name, pred_threshold=0.8, save=True)
candidate_map.add_data(data=blobs, name='Candidate Blobs')

In [ ]:
# Plot blob locations on a satellite base image
candidate_map